In [9]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
# features.shape,labels.shape

In [10]:
def load_array(data_arrays,batch_size,is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size = 10
data_iter = load_array((features,labels),batch_size)

In [11]:
from torch import nn

# 定义网络
net = nn.Sequential(nn.Linear(2,1))
# 定义参数
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)
# 定义损失函数
loss = nn.MSELoss()
# 定义优化
optimizer = torch.optim.SGD(net.parameters(),lr=0.03)

# 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000271
epoch 2, loss 0.000101
epoch 3, loss 0.000101


In [12]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0003,  0.0008])
b的估计误差： tensor([0.0004])


##### T1.如果将小批量的总损失替换为小批量损失的平均值，需要如何更改学习率？
如果将小批量的总损失替换为小批量损失的平均值，则需要将学习率乘以批量大小。这是因为在计算梯度时，我们使用了小批量中所有样本的信息。因此，如果我们将小批量的总损失替换为小批量损失的平均值，则相当于将每个样本的梯度除以批量大小。因此，我们需要将学习率乘以批量大小，以保持相同的更新步长。

##### T2.查看深度学习框架文档，它们提供了哪些损失函数和初始化方法？用Huber损失代替原损失结果会是什么样？
PyTorch文档地址：https://pytorch.ac.cn/docs/stable/index.html<br>
损失函数：https://pytorch.ac.cn/docs/stable/nn.html#loss-functions<br>
初始化方法：https://pytorch.ac.cn/docs/stable/nn.init.html<br>


In [17]:
from torch import nn

# 定义网络
net = nn.Sequential(nn.Linear(2,1))
# 定义参数
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)
# 定义损失函数
loss = nn.HuberLoss()
# 定义优化
optimizer = torch.optim.SGD(net.parameters(),lr=0.03)

# 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

epoch 1, loss 2.228370
epoch 2, loss 0.482176
epoch 3, loss 0.002280
w的估计误差： tensor([ 0.0175, -0.0533])
b的估计误差： tensor([0.0383])


In [19]:
# test 3:访问线性回归的梯度
w_grad = net[0].weight.grad
print("w的梯度为：",w_grad)
b_grad = net[0].bias.grad
print("b的梯度为：",b_grad)



w的梯度为： tensor([[-0.0120,  0.0335]])
b的梯度为： tensor([-0.0477])
